In [264]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [265]:



df = pd.read_csv("train.csv")


print(df.head())


   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [266]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [267]:
data = df.select_dtypes(include=['number']).to_numpy()
print(np.shape(data))

(1460, 38)


In [268]:




# Impute missing values with column means
imputer = SimpleImputer(strategy="mean")
z = imputer.fit_transform(data)

# Separate features and target
y = z[:, -1]   # assuming last column is SalePrice
x = z[:, 1:-1]

print("Any NaN left in x?", np.isnan(x).any())
print("Any NaN left in y?", np.isnan(y).any())




Any NaN left in x? False
Any NaN left in y? False


In [269]:

scaler = StandardScaler()
x = scaler.fit_transform(x)

y_original = y.copy()  # Keep original for inverse transformation
y_scaler = StandardScaler()
y = y_scaler.fit_transform(y.reshape(-1, 1)).flatten()

In [270]:

n = x.shape[1]
init_w = np.zeros(n)
init_b = 0
print(x.shape)
print(y.shape)


(1460, 36)
(1460,)


In [271]:
print(init_w.shape)
print(x)



(36,)
[[ 0.07337496 -0.22937175 -0.20714171 ... -0.08768781 -1.5991111
   0.13877749]
 [-0.87256276  0.4519361  -0.09188637 ... -0.08768781 -0.48911005
  -0.61443862]
 [ 0.07337496 -0.09311018  0.07347998 ... -0.08768781  0.99089135
   0.13877749]
 ...
 [ 0.30985939 -0.18395123 -0.14781027 ...  4.95311151 -0.48911005
   1.64520971]
 [-0.87256276 -0.09311018 -0.08016039 ... -0.08768781 -0.8591104
   1.64520971]
 [-0.87256276  0.22483348 -0.05811155 ... -0.08768781 -0.1191097
   0.13877749]]


In [272]:

m = len(x)
print(m)
y_pred = np.zeros(m)
def predict(X, w, b):
    return np.dot(X, w) + b
            


1460


In [273]:
def calc_cost(actual,prediction):
    c=0
    cost = 0
    for i in range(m):
        c+= (actual[i] - prediction[i])**2
    cost = (1/(2*m))*c
    return cost

In [274]:
y_pred = predict(x, init_w, init_b)

print(y)
print(y_pred)

J = calc_cost(y, y_pred)

[ 0.34727322  0.00728832  0.53615372 ...  1.07761115 -0.48852299
 -0.42084081]
[0. 0. 0. ... 0. 0. 0.]


Formula for Gradient:

djdw = 1/m sum(((wx_i + b) - yi)xi)
djdb = 1/m sum(((wx_i + b) - yi ))

In [275]:
def compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_dw, dj_db

In [276]:
cost_history = []
alpha = 0.01

for i in range(1000):
    dj_dw, dj_db = compute_gradient(x, y, init_w, init_b)
    init_w -= alpha * dj_dw
    init_b -= alpha * dj_db
    
    if i % 100 == 0:  
        cost = calc_cost(y, predict(x, init_w, init_b))
        cost_history.append(cost)
        print(f"Iteration {i}: Cost = {cost:.2f}")

Iteration 0: Cost = 0.45
Iteration 100: Cost = 0.10
Iteration 200: Cost = 0.10
Iteration 300: Cost = 0.10
Iteration 400: Cost = 0.09
Iteration 500: Cost = 0.09
Iteration 600: Cost = 0.09
Iteration 700: Cost = 0.09
Iteration 800: Cost = 0.09
Iteration 900: Cost = 0.09


In [277]:
final_cost = calc_cost(y, predict(x, init_w, init_b))
print("Final cost:", final_cost)

Final cost: 0.09357731633209497


In [278]:
y_pred_scaled = predict(x, init_w, init_b)
y_pred_original = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

rmse = np.sqrt(np.mean((y_original - y_pred_original)**2))
print(f"RMSE in original dollars: ${rmse:,.2f}")


print("\nSample predictions vs actual:")
for i in range(5):
    print(f"Predicted: ${y_pred_original[i]:,.2f}, Actual: ${y_original[i]:,.2f}")

RMSE in original dollars: $34,356.15

Sample predictions vs actual:
Predicted: $228,858.54, Actual: $208,500.00
Predicted: $193,041.95, Actual: $181,500.00
Predicted: $224,157.55, Actual: $223,500.00
Predicted: $197,705.73, Actual: $140,000.00
Predicted: $290,397.68, Actual: $250,000.00
